In [ ]:
from allennlp.predictors.predictor import Predictor

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/634M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import pandas as pd
df=pd.read_csv('/content/processed_data')
qa_sets = []
for i in range(0, len(df), 10):
    subset = df[i:i+10]
    qa_set = [f"{question}. {answer}" for question, answer in zip(subset['Questions'], subset['Answers'])]
    qa_sets.append(qa_set)

In [ ]:
qa_sets

[['is the photo in color. yes',
  'is it a professional photo. no',
  'is it well lit. no',
  'is it daytime. i don t see windows',
  'doe this look like an adult bedroom. maybe',
  'is the room clean. no',
  'can you tell what kind of computer it is. no not really',
  'is it a flat screen. yes',
  'what s the desk made out of. cheap plastic or wood',
  'is there a computer chair. yes'],
 ['is this in a park. yes i believe it is',
  'are there others around. no she is alone',
  'doe she have a collection bucket. no',
  'is her hair long. yes pretty long',
  'is she wearing a dress. i don t think so hard to tell',
  'doe she have shoe on. yes flip flops',
  'is there grass nearby. yes everywhere',
  'is it a sunny day. yes',
  'are there tree. in the background there are trees',
  'is the guitar new. i don t think so'],
 ['is this in the bathroom. yes',
  'doe he look sad. he looks puzzled',
  'how old is he. my best guess would be between 18 and 25',
  'puzzled like he drank to much la

In [ ]:
c =[]
for i in range(6,11):
  d=[]
  for j in qa_sets[i]:
    d.append(j)
  c.append(d)
c

[['what color is the jet. white and blue',
  'is it a commercial plane. yes',
  'can the sky be seen. yes',
  'are there any cloud. no',
  'do you see any other plane in the picture. no',
  'can you see anything else other than the plane. no',
  'is the plane crashing. no',
  'is the plane above any body of water. yes',
  'are those body of water lake. possibly',
  'is there any text on the plane. not that i can see'],
 ['do you see a lot of people. just 3',
  'what is the tennis player wearing. white tennis dress',
  'what color is her tennis racket. black',
  'is she wearing a hat. a visor',
  'is she close to the net. no',
  'do you see another player. no',
  'do you see a tennis bag. no',
  'do you see tree. no',
  'do you see anything next to her. nothing',
  'doe she look mad. no'],
 ['what kind of food. looks like fried chicken and some rice and sauce',
  'what color are the plate. they are orange',
  'what color are the bowl. also orange',
  'are there any drink. no drinks are 

In [ ]:
import string

string_result = ''
a = []

for i in c:
    string_result = ' / '.join(i)
    #string_result = string_result.translate(str.maketrans('', '', string.punctuation))
    a.append(string_result )

a

['what color is the jet. white and blue / is it a commercial plane. yes / can the sky be seen. yes / are there any cloud. no / do you see any other plane in the picture. no / can you see anything else other than the plane. no / is the plane crashing. no / is the plane above any body of water. yes / are those body of water lake. possibly / is there any text on the plane. not that i can see',
 'do you see a lot of people. just 3 / what is the tennis player wearing. white tennis dress / what color is her tennis racket. black / is she wearing a hat. a visor / is she close to the net. no / do you see another player. no / do you see a tennis bag. no / do you see tree. no / do you see anything next to her. nothing / doe she look mad. no',
 'what kind of food. looks like fried chicken and some rice and sauce / what color are the plate. they are orange / what color are the bowl. also orange / are there any drink. no drinks are visible / are there any people. no people are visible / what color i

In [ ]:
ground_truth_clusters = [
    [('jet', 'color'), ('plane', 'object')],
    [('tennis player', 'object'), ('she', 'person')],
    [('plate', 'object'), ('table', 'object')],
    [('toilet', 'object'), ('floor', 'object')],
    [('truck', 'object'), ('picture', 'object')]
]

print(ground_truth_clusters)

[[('jet', 'color'), ('plane', 'object')], [('tennis player', 'object'), ('she', 'person')], [('plate', 'object'), ('table', 'object')], [('toilet', 'object'), ('floor', 'object')], [('truck', 'object'), ('picture', 'object')]]


In [ ]:
def get_coreference_clusters(document):
    pred = predictor.predict(document=document)
    clusters = pred['clusters']
    document = pred['document']

    n = 0
    doc = {}
    for obj in document:
        doc.update({n: obj})
        n += 1

    clus_all = []
    cluster = []

    for i in range(len(clusters)):
        one_cl = clusters[i]
        for count in range(len(one_cl)):
            obj = one_cl[count]
            for num in range(obj[0], obj[1]+1):
                for n in doc:
                    if num == n:
                        cluster.append(doc[n])
        clus_all.append(cluster)
        cluster = []

    return clus_all

In [ ]:
clus_full = []
for i in range(len(a)):
  pred = predictor.predict(
         document= a[i]
  )
  clusters = pred['clusters']
  document = pred['document']
  n = 0
  doc = {}
  for obj in document:
      doc.update({n :  obj})
      n = n+1
  print(f"doc {i+1} : {doc}")
  print("")
  clus_all = []
  cluster = []
  clus_one = {}
  for i in range(0, len(clusters)):
      one_cl = clusters[i]
      for count in range(0, len(one_cl)):
          obj = one_cl[count]
          for num in range((obj[0]), (obj[1]+1)):
              for n in doc:
                  if num == n:
                      cluster.append(doc[n])
      clus_all.append(cluster)
      cluster = []
  clus_full.append(clus_all)
  for i in range(len(clus_all)):
    print(f"Cluster {i+1} : {clus_all[i]}")
  print('---------------------------------------------------------------------------------------------------------------------------------------------------')
  print('')

doc 1 : {0: 'what', 1: 'color', 2: 'is', 3: 'the', 4: 'jet', 5: '.', 6: 'white', 7: 'and', 8: 'blue', 9: '/', 10: 'is', 11: 'it', 12: 'a', 13: 'commercial', 14: 'plane', 15: '.', 16: 'yes', 17: '/', 18: 'can', 19: 'the', 20: 'sky', 21: 'be', 22: 'seen', 23: '.', 24: 'yes', 25: '/', 26: 'are', 27: 'there', 28: 'any', 29: 'cloud', 30: '.', 31: 'no', 32: '/', 33: 'do', 34: 'you', 35: 'see', 36: 'any', 37: 'other', 38: 'plane', 39: 'in', 40: 'the', 41: 'picture', 42: '.', 43: 'no', 44: '/', 45: 'can', 46: 'you', 47: 'see', 48: 'anything', 49: 'else', 50: 'other', 51: 'than', 52: 'the', 53: 'plane', 54: '.', 55: 'no', 56: '/', 57: 'is', 58: 'the', 59: 'plane', 60: 'crashing', 61: '.', 62: 'no', 63: '/', 64: 'is', 65: 'the', 66: 'plane', 67: 'above', 68: 'any', 69: 'body', 70: 'of', 71: 'water', 72: '.', 73: 'yes', 74: '/', 75: 'are', 76: 'those', 77: 'body', 78: 'of', 79: 'water', 80: 'lake', 81: '.', 82: 'possibly', 83: '/', 84: 'is', 85: 'there', 86: 'any', 87: 'text', 88: 'on', 89: 'the'

In [ ]:
def evaluate_accuracy(ground_truth_clusters, predicted_clusters):
    correct_predictions = 0
    total_predictions = 0

    for predicted_cluster, true_cluster in zip(predicted_clusters, ground_truth_clusters):
        total_predictions += len(predicted_cluster)

        predicted_mentions = set([mention.lower() for sublist in predicted_cluster for mention in sublist])

        true_mentions = set([mention.lower() for sublist in true_cluster for mention in sublist])

        correct_predictions += len(predicted_mentions.intersection(true_mentions))

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

In [ ]:


# Preprocess the sample texts
preprocessed_data = a

# Perform coreference resolution
predicted_clusters = [get_coreference_clusters(text) for text in preprocessed_data]

# Evaluate accuracy
accuracy = evaluate_accuracy(ground_truth_clusters, predicted_clusters)
print(f"Accuracy: {accuracy}%")


Accuracy: 60.0%
